In [1]:
import numpy as np

In [2]:
class BasicNeuron:
    """
    A basic artificial neuron implementation that mimics biological neurons.

    The neuron receives inputs, applies weights, adds bias, and produces an output
    through an activation function - just like neurons in our brain!
    """

    def __init__(self, num_inputs, activation_function='sigmoid'):
        """
        Initialize the neuron with random weights and bias.

        Args:
            num_inputs: Number of input connections to this neuron
            activation_function: Type of activation function ('sigmoid', 'relu', 'tanh', 'linear')
        """
        # Initialize weights randomly between -1 and 1
        # Each input gets its own weight - this determines how important each input is
        self.weights = np.random.uniform(-1, 1, num_inputs)

        # Initialize bias - this shifts the activation function left or right
        # Bias helps the neuron fire even when inputs are small
        self.bias = np.random.uniform(-1, 1)

        # Store the activation function type
        if activation_function not in ['sigmoid', 'relu', 'tanh', 'linear']:
            raise ValueError(f"Unsupported activation function: {activation_function}")
        self.activation_function = activation_function

        # Store the number of inputs for validation
        self.num_inputs = num_inputs

        # print(f"Neuron created with {num_inputs} inputs")
        # print(f"Initial weights: {self.weights}")
        # print(f"Initial bias: {self.bias}")
        # print(f"Activation function: {activation_function}")

    def sigmoid(self, x):
        """
        Sigmoid activation function: f(x) = 1 / (1 + e^(-x))

        - Outputs values between 0 and 1
        - Smooth, differentiable curve
        - Good for binary classification problems
        """
        # Clip x to prevent overflow in exponential
        x = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """Derivative of the sigmoid function."""
        # Derivative of sigmoid(x) is sigmoid(x) * (1 - sigmoid(x))
        # We can use the output of the sigmoid function itself for this
        s = self.sigmoid(x)
        return s * (1 - s)

    def relu(self, x):
        """ReLU activation function: f(x) = max(0, x)"""
        return np.maximum(0, x)

    def relu_derivative(self, x):
        """Derivative of the ReLU function."""
        return np.where(x > 0, 1, 0)

    def tanh(self, x):
        """Tanh activation function: f(x) = tanh(x)"""
        return np.tanh(x)

    def tanh_derivative(self, x):
        """Derivative of the Tanh function."""
        # Derivative of tanh(x) is 1 - tanh(x)^2
        return 1 - np.tanh(x)**2

    def linear(self, x):
        """Linear activation function: f(x) = x"""
        return x

    def linear_derivative(self, x):
        """Derivative of the Linear function."""
        return 1

    def get_activation_derivative(self, z):
        """Returns the derivative of the chosen activation function for a given net input z."""
        if self.activation_function == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif self.activation_function == 'relu':
            return self.relu_derivative(z)
        elif self.activation_function == 'tanh':
            return self.tanh_derivative(z)
        elif self.activation_function == 'linear':
            return self.linear_derivative(z)
        else:
            raise ValueError(f"Derivative not implemented for {self.activation_function}")


    def forward(self, inputs):

        # Convert inputs to numpy array for easier computation
        inputs = np.array(inputs)

        # Validate input size
        if len(inputs) != self.num_inputs:
            raise ValueError(f"Expected {self.num_inputs} inputs, got {len(inputs)}")

        # Step 1 & 2: Weighted sum (dot product of inputs and weights)
        # This is like: w1*x1 + w2*x2 + w3*x3 + ... + wn*xn
        weighted_sum = np.dot(inputs, self.weights)

        # Step 3: Add bias
        # Bias allows the neuron to fire even when inputs are zero
        z = weighted_sum + self.bias

        # Step 4: Apply activation function
        if self.activation_function == 'sigmoid':
            output = self.sigmoid(z)
        elif self.activation_function == 'relu':
            output = self.relu(z)
        elif self.activation_function == 'tanh':
            output = self.tanh(z)
        elif self.activation_function == 'linear':
            output = self.linear(z)
        else:
            raise ValueError(f"Unknown activation function: {self.activation_function}")

        # Store intermediate values for educational purposes
        self.last_inputs = inputs
        self.last_weighted_sum = weighted_sum
        self.last_z = z # Store z (net input before activation) for derivative calculation
        self.last_output = output

        return output

    def update_weights(self, new_weights, new_bias=None):
        if len(new_weights) != self.num_inputs:
            raise ValueError(f"Expected {self.num_inputs} weights, got {len(new_weights)}")

        self.weights = np.array(new_weights)

        if new_bias is not None:
            self.bias = new_bias

        # print(f"Weights updated to: {self.weights}")
        # print(f"Bias updated to: {self.bias}")

    def get_details(self, verbose=True):
        if hasattr(self, 'last_inputs'):
            if verbose:
                print("\n--- Neuron Computation Details ---")
                print(f"Inputs: {self.last_inputs}")
                print(f"Weights: {self.weights}")
                print(f"Weighted sum: {self.last_weighted_sum:.4f}")
                print(f"Bias: {self.bias:.4f}")
                print(f"z (weighted sum + bias): {self.last_z:.4f}")
                print(f"Final output: {self.last_output:.4f}")
        else:
            if verbose:
                print("No computation has been performed yet!")


In [ ]:
#correlation

def correlation_learning(inputs, outputs, learning_rate=1.0):
    """
    Implements the Correlation Learning Law similar to Hebbian learning.
    - inputs: shape (num_patterns, num_inputs)
    - outputs: shape (num_patterns,) or (num_patterns, num_outputs)
    """
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')
    # Initialize weights to zeros for simplicity
    neuron.update_weights(np.zeros(num_inputs), 0.0)

    num_patterns = inputs.shape[0]

    print(f"\n--- Correlation Learning ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for i in range(num_patterns):
        x = inputs[i]
        y_desired = outputs[i]

        # Correlation learning rule:
        # delta_w = learning_rate * (x * y_desired)
        # similar to Hebbian, but emphasizes correlation between x and y
        delta_weights = learning_rate * np.outer(x, [y_desired]).flatten()

        new_weights = neuron.weights + delta_weights
        new_bias = neuron.bias  # correlation law typically ignores bias update

        neuron.update_weights(new_weights, new_bias)

        print(f"Pattern {i+1}: Input={x}, Desired Output={y_desired}")
        print(f"  Delta Weights: {delta_weights}")
        print(f"  Updated Weights: {neuron.weights}, Updated Bias: {neuron.bias}")

    print(f"Final Weights (Correlation): {neuron.weights}, Final Bias: {neuron.bias}")
    return neuron.weights

In [10]:
Correlation_inputs = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])
Correlation_outputs = np.array([
    0,  
    0, 
    1,  
    0   
])
correlation_learning(Correlation_inputs, Correlation_outputs)


--- Correlation Learning ---
Initial Weights: [0. 0.], Initial Bias: 0.0
Pattern 1: Input=[1 0], Desired Output=0
  Delta Weights: [0. 0.]
  Updated Weights: [0. 0.], Updated Bias: 0.0
Pattern 2: Input=[0 1], Desired Output=0
  Delta Weights: [0. 0.]
  Updated Weights: [0. 0.], Updated Bias: 0.0
Pattern 3: Input=[1 1], Desired Output=1
  Delta Weights: [1. 1.]
  Updated Weights: [1. 1.], Updated Bias: 0.0
Pattern 4: Input=[0 0], Desired Output=0
  Delta Weights: [0. 0.]
  Updated Weights: [1. 1.], Updated Bias: 0.0
Final Weights (Correlation): [1. 1.], Final Bias: 0.0


array([1., 1.])

In [ ]:
#outstar

def outstar_learning(inputs, outputs, learning_rate=0.1, max_epochs=100, tolerance=1e-6):
    """
    Implements Outstar Learning with convergence.
    - inputs: shape (num_patterns, num_inputs)
    - outputs: shape (num_patterns, num_outputs) or (num_patterns,)
    - learning_rate: step size for updates
    - max_epochs: maximum number of epochs
    - tolerance: convergence threshold
    """
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')
    
    # Initialize weights to zeros
    neuron.update_weights(np.zeros(num_inputs), 0.0)
    
    print("\n--- Outstar Learning with Convergence ---")
    print(f"Initial Weights: {neuron.weights}")
    
    for epoch in range(max_epochs):
        total_change = 0.0
        
        # Outstar rule is teacher-driven → we use outputs directly
        for i in range(inputs.shape[0]):
            y_desired = outputs[i]
            
            # Update rule: w <- w + η (y_desired - w)
            delta_w = learning_rate * (y_desired - neuron.weights)
            new_weights = neuron.weights + delta_w
            
            total_change += np.linalg.norm(delta_w)
            
            neuron.update_weights(new_weights, neuron.bias)  # bias unchanged
        
        # print(f"Epoch {epoch+1}: Weights={neuron.weights}, Change={total_change:.6f}")
        
        # Convergence check
        if total_change < tolerance:
            print(f"Converged after {epoch+1} epochs.")
            break
    
    print(f"Final Weights: {neuron.weights}")
    return neuron.weights


In [7]:
Outstar_inputs = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])
Outstar_outputs = np.array([
    1,  
    1, 
    1,  
    0   
])
outstar_learning(Outstar_inputs, Outstar_outputs)


--- Outstar Learning with Convergence ---
Initial Weights: [0. 0.]
Final Weights: [0.7092178 0.7092178]


array([0.7092178, 0.7092178])

In [ ]:
#competitive

def instar_learning(inputs, num_neurons=2, learning_rate=0.5, epochs=10):
    """
    Implements the Instar Learning Law (Grossberg) with Winner-Take-All.
    Used for competitive learning / clustering. Neurons compete to respond to an input pattern.
    The winner's weights are moved closer to the input pattern.
    Update rule for winning neuron j: Δw_j = η * (x - w_j)
    """
    num_inputs = inputs.shape[1]
    # Create a layer of neurons
    neurons = [BasicNeuron(num_inputs, activation_function='linear') for _ in range(num_neurons)]

    print(f"\n--- Instar (Winner-Take-All) Learning Law ---")
    for i, n in enumerate(neurons):
        print(f"Initial Weights for Neuron {i+1}: {n.weights}")

    for epoch in range(epochs):
        print(f"\n--- Epoch {epoch+1} ---")
        for x in inputs:
            # 1. Competition: Find the winning neuron (closest weight vector)
            # The one with the highest dot product (net input) wins.
            net_inputs = [np.dot(n.weights, x) for n in neurons]
            winner_index = np.argmax(net_inputs)
            winner_neuron = neurons[winner_index]

            # 2. Update winner's weights
            delta_weights = learning_rate * (x - winner_neuron.weights)
            new_weights = winner_neuron.weights + delta_weights
            winner_neuron.update_weights(new_weights)
            
            print(f"Input: {x}, Winner: Neuron {winner_index+1}, Updated Weights: {winner_neuron.weights}")

    print("\nFinal Weights (Instar):")
    for i, n in enumerate(neurons):
        print(f"Neuron {i+1}: {n.weights}")
    return [n.weights for n in neurons]


In [ ]:
competitive_inputs = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])
instar_learning(competitive_inputs)


--- Instar (Winner-Take-All) Learning Law ---
Initial Weights for Neuron 1: [ 0.42563814 -0.02468607]
Initial Weights for Neuron 2: [ 0.92569105 -0.9065473 ]

--- Epoch 1 ---
Input: [1 0], Winner: Neuron 2, Updated Weights: [ 0.96284553 -0.45327365]
Input: [0 1], Winner: Neuron 1, Updated Weights: [0.21281907 0.48765696]
Input: [1 1], Winner: Neuron 1, Updated Weights: [0.60640953 0.74382848]
Input: [0 0], Winner: Neuron 1, Updated Weights: [0.30320477 0.37191424]

--- Epoch 2 ---
Input: [1 0], Winner: Neuron 2, Updated Weights: [ 0.98142276 -0.22663682]
Input: [0 1], Winner: Neuron 1, Updated Weights: [0.15160238 0.68595712]
Input: [1 1], Winner: Neuron 1, Updated Weights: [0.57580119 0.84297856]
Input: [0 0], Winner: Neuron 1, Updated Weights: [0.2879006  0.42148928]

--- Epoch 3 ---
Input: [1 0], Winner: Neuron 2, Updated Weights: [ 0.99071138 -0.11331841]
Input: [0 1], Winner: Neuron 1, Updated Weights: [0.1439503  0.71074464]
Input: [1 1], Winner: Neuron 2, Updated Weights: [0.99

[array([4.39301446e-06, 3.33334678e-01]), array([0.99999972, 0.66665304])]

In [6]:
#reinforcement

def reinforcement_learning(inputs, rewards, learning_rate=1.0):
    """
    Implements a simple Reinforcement Learning rule.
    - inputs: shape (num_patterns, num_inputs)
    - rewards: shape (num_patterns,) reinforcement signals (+1 for reward, -1 for punishment, 0 for neutral)
    """
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')
    # Initialize weights to zeros for simplicity
    neuron.update_weights(np.zeros(num_inputs), 0.0)

    num_patterns = inputs.shape[0]

    print(f"\n--- Reinforcement Learning ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for i in range(num_patterns):
        x = inputs[i]
        r = rewards[i]

        # Current neuron output
        y = neuron.forward(x)

        delta_weights = learning_rate * r * x

        new_weights = neuron.weights + delta_weights
        new_bias = neuron.bias  # bias is often left unchanged here

        neuron.update_weights(new_weights, new_bias)

        print(f"Pattern {i+1}: Input={x}, Reward={r}, Output={y}")
        print(f"  Delta Weights: {delta_weights}")
        print(f"  Updated Weights: {neuron.weights}, Updated Bias: {neuron.bias}")

    print(f"Final Weights: {neuron.weights}, Final Bias: {neuron.bias}")

    # Test with last input
    test_output = neuron.forward(inputs[-1])
    print(f"Test output for last pattern: {test_output}")

    return neuron.weights


In [ ]:
#boltzmann

def boltzmann_learning(inputs, outputs, learning_rate=0.1, epochs=10):
    """
    Implements a simplified Boltzmann Learning Law using the BasicNeuron class.
    
    Args:
        inputs: shape (num_patterns, num_inputs)
        outputs: shape (num_patterns,) - desired target outputs (clamped phase)
        learning_rate: learning rate (η)
        epochs: number of training iterations

    Returns:
        Final learned weights
    """
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='sigmoid')  # sigmoid is typical for Boltzmann machines
    
    # Initialize weights small
    neuron.update_weights(np.random.uniform(-0.1, 0.1, num_inputs), 0.0)

    print("\n--- Boltzmann Learning ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}:")
        
        for i in range(inputs.shape[0]):
            x = inputs[i]
            y_clamped = outputs[i]  # "data" expectation (clamped phase)

            # --- Clamped phase (desired output) ---
            clamped_corr = np.outer(x, [y_clamped]).flatten()

            # --- Free phase (model output) ---
            y_free = neuron.forward(x)  # model’s own prediction
            free_corr = np.outer(x, [y_free]).flatten()

            # --- Weight update rule (Boltzmann Learning) ---
            delta_weights = learning_rate * (clamped_corr - free_corr)

            new_weights = neuron.weights + delta_weights
            neuron.update_weights(new_weights, neuron.bias)

            print(f"  Pattern {i+1}: Input={x}, Desired={y_clamped}, Free={y_free:.4f}")
            print(f"    Clamped Corr: {clamped_corr}")
            print(f"    Free Corr:    {free_corr}")
            print(f"    Delta W:      {delta_weights}")
            print(f"    Updated W:    {neuron.weights}")

    print(f"\nFinal Weights (Boltzmann): {neuron.weights}, Final Bias: {neuron.bias}")
    return neuron.weights


In [ ]:
#widrow hoff lms

def widrow_hoff_learning(inputs, outputs, learning_rate=0.1, epochs=10):
    """
    Implements the Widrow-Hoff (LMS/Delta Rule) learning law.
    
    Args:
        inputs: shape (num_patterns, num_inputs)
        outputs: shape (num_patterns,) - desired outputs
        learning_rate: learning rate (η)
        epochs: number of training iterations
    
    Returns:
        Final learned weights and bias
    """
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')  # LMS is usually linear
    
    # Initialize weights small
    neuron.update_weights(np.random.uniform(-0.1, 0.1, num_inputs), 0.0)

    print("\n--- Widrow-Hoff LMS Learning ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}:")
        
        for i in range(inputs.shape[0]):
            x = inputs[i]
            y_desired = outputs[i]

            # Forward pass (actual output)
            y_actual = neuron.forward(x)

            # Compute error
            error = y_desired - y_actual

            # Weight update (Delta Rule)
            delta_weights = learning_rate * error * x
            delta_bias = learning_rate * error

            new_weights = neuron.weights + delta_weights
            new_bias = neuron.bias + delta_bias

            neuron.update_weights(new_weights, new_bias)

            print(f"  Pattern {i+1}: Input={x}, Desired={y_desired}, Actual={y_actual:.4f}")
            print(f"    Error: {error:.4f}")
            print(f"    ΔW: {delta_weights}, ΔB: {delta_bias:.4f}")
            print(f"    Updated W: {neuron.weights}, Updated B: {neuron.bias:.4f}")

    print(f"\nFinal Weights (Widrow-Hoff): {neuron.weights}, Final Bias: {neuron.bias}")
    return neuron.weights, neuron.bias
